# 2 - Process spectral data

Loads adjusted data and formats for spectral analysis.

## Imports

Necessary modules for analysis.

In [1]:
# import modules

import xarray as xr
import numpy as np
import scipy.signal as sig
for i in range(2):
    %matplotlib notebook

In [2]:
# import data

adcp = 'Shoulder'     # Slope(2013,2014,2017,2018), Axis75(2013,2014), Axis55(2017,2018), Shoulder
year = 2013
if adcp == 'Shoulder':
    adcp2 = 'Slope'
    ds_in = xr.open_dataset(f'../Data/data/adj/adj_{adcp2}_1min_{year}_0.nc')
    n_seg = ds_in.n_seg
    if n_seg > 1:
        ds = [ds_in]
        for i in range(n_seg):
            if i > 0:
                ds_temp = xr.open_dataset(f'../Data/data/adj/adj_{adcp2}_1min_{year}_{i}.nc')
                ds.append(ds_temp)
    elif n_seg == 1:
        ds = [ds_in]
else:
    ds_in = xr.open_dataset(f'../Data/data/adj/adj_{adcp}_{year}_0.nc')
    n_seg = ds_in.n_seg
    if n_seg > 1:
        ds = [ds_in]
        for i in range(n_seg):
            if i > 0:
                ds_temp = xr.open_dataset(f'../Data/data/adj/adj_{adcp}_{year}_{i}.nc')
                ds.append(ds_temp)
    elif n_seg == 1:
        ds = [ds_in]
    
#print(ds)

In [3]:
# extract variables

t_stamp = ds[0].t_stamp
depth = ds[0].depth.values
d = ds[0].d
start_date = ds[0].start_date
end_date = ds[0].end_date

# get WKB scale for site specific depth range

scaling_array = np.load('../project/archive/N2/scaling_array.npy')
GM_depths = scaling_array[0]                                 # depths range from -4 to -924 metres
GM_scale = scaling_array[1]
int_scale = np.interp(depth,-GM_depths,GM_scale)

## Format spectrogram data
Removes the mean from each depth. Performs spectrogram process with optimised parameters for visual analysis of spectra.

In [4]:
# spectra data adjustments & Welch parameters

time_total = 0
for i in range(n_seg):
    time_total += len(ds[i].t_seg)
print('Total time length:',time_total)

if adcp == 'Shoulder':
    # set Welch parameters for PSD
    fs = 1.6667e-2                # 60 samples per HOUR, 1.67e-2 per SECOND
    win = 'hann'                  # optimal window for averaging
    nps = 2**14                    # find optimal window for nperseg (1024 ~10 days)
    overlap = 0.5*nps             # 50% overlap, default   
    # set Welch parameters for spectrograms and depth-band plots
    nps_s = 2**13                   # find optimal window for nperseg (512 ~5 days)
    overlap_s = 0.5*nps_s            # 50% overlap, default   
    # set Welch parameters for continuum analysis
    nps_c = 2**10                   # find optimal window for nperseg (256 ~2.5 days)
    overlap_c = 0.5*nps_c         # 50% overlap, default   
    # set Welch parameters for tidal analysis
    nps_SN = 2**13                    # find optimal window for nperseg (2048 ~20 days)
    overlap_SN = 0.8*nps_SN         # 50% overlap, default  
    
else:
    # set Welch parameters for PSD

    fs = 1.1111e-3                # 4 samples per HOUR, 1.11e-3 per SECOND
    win = 'hann'                  # optimal window for averaging
    nps = 1024                    # find optimal window for nperseg (1024 ~10 days)
    #nps = 4096                   # fine spectra
    overlap = nps / 2             # 50% overlap, default   

    # set Welch parameters for spectrograms and depth-band plots

    nps_s = 512                   # find optimal window for nperseg (512 ~5 days)
    overlap_s = 0.5*nps_s            # 50% overlap, default   

    # set Welch parameters for continuum analysis

    nps_c = 256                   # find optimal window for nperseg (256 ~2.5 days)
    overlap_c = 0.5*nps_c         # 50% overlap, default   

    # set Welch parameters for tidal analysis

    nps_SN = 512                    # find optimal window for nperseg (2048 ~20 days)
    overlap_SN = 0.8*nps_SN         # 50% overlap, default   

# remove short segments

t_short = []
for i in range(n_seg):
    if len(ds[i].t_seg) < nps:
        t_short.append(i)
for i in sorted(t_short, reverse=True):
    del ds[i]
n_seg = n_seg - len(t_short)

time_total = 0
for i in range(n_seg):
    time_total += len(ds[i].t_seg)
print(len(t_short),'segment(s) too short, new total time length:',time_total)

Total time length: 514311
0 segment(s) too short, new total time length: 514311


In [5]:
# remove mean at each depth

um,vm,uh,vh = [],[],[],[]
for i in range(n_seg):
    um_seg,vm_seg,uh_seg,vh_seg = [],[],[],[]
    uorig_temp = ds[i].uorig.values
    vorig_temp = ds[i].vorig.values
    uh_temp = ds[i].uorig.values - ds[i].ulp.values   # high pass for tidal analysis
    vh_temp = ds[i].vorig.values - ds[i].vlp.values
    for j in range(d):
        umtemp = np.copy(uorig_temp[:,j]) - np.nanmean(uorig_temp[:,j])
        vmtemp = np.copy(vorig_temp[:,j]) - np.nanmean(vorig_temp[:,j])
        um_seg.append(umtemp)
        vm_seg.append(vmtemp)
        uhtemp = np.copy(uh_temp[:,j]) - np.nanmean(uh_temp[:,j])
        vhtemp = np.copy(vh_temp[:,j]) - np.nanmean(vh_temp[:,j])
        uh_seg.append(uhtemp)
        vh_seg.append(vhtemp)
    um.append(um_seg)     # list[segment][depth][time]
    vm.append(vm_seg)     # 0 is upper depth index
    uh.append(uh_seg)     # list[segment][depth][time]
    vh.append(vh_seg)     # 0 is upper depth index

/var/folders/2_/jy6ccs_j16l04nxn4kmnxmlr0000gn/T/ipykernel_10096/1424463972.py:15: RuntimeWarning: Mean of empty slice
  uhtemp = np.copy(uh_temp[:,j]) - np.nanmean(uh_temp[:,j])
/var/folders/2_/jy6ccs_j16l04nxn4kmnxmlr0000gn/T/ipykernel_10096/1424463972.py:16: RuntimeWarning: Mean of empty slice
  vhtemp = np.copy(vh_temp[:,j]) - np.nanmean(vh_temp[:,j])


In [6]:
# define cross-spectra function using spectrograms

def spectrocross(u,v,fs,window,nperseg,noverlap,nfft,detrend,return_onesided,scaling,axis,mode):
    freqs, time, Sxy = sig._spectral_helper(u, v, fs, window, nperseg,
                                            noverlap, nfft, detrend,
                                            return_onesided, scaling, axis,
                                            mode='psd')        # get this from scipy spectral.py in pkgs (add name to import list)
    return freqs, time, Sxy

In [7]:
# spectrograms at each depth (PSD at each depth in time-steps of nps)

Sduh,Sdvh,spect_SN,specf_SN,cw_SN,ccw_SN = [],[],[],[],[],[]
spect,specf,spect_s,specf_s,Sxu,Sxv,cw,ccw,Sdu,Sdv = [],[],[],[],[],[],[],[],[],[]
cw_s, ccw_s = [],[]
spect_c,specf_c,Scu,Scv,cw_c,ccw_c = [],[],[],[],[],[]

for i in range(n_seg): 
    specf_SN_temp,spect_SN_temp,Sdvh_temp,Sduh_temp,cw_SN_temp,ccw_SN_temp = [],[],[],[],[],[]
    spect_temp,specf_temp,cw_temp,ccw_temp,spect_s_temp,specf_s_temp,Sxu_temp,Sxv_temp,Sdu_temp,Sdv_temp=[],[],[],[],[],[],[],[],[],[]
    cw_s_temp,ccw_s_temp = [],[]
    spect_c_temp,specf_c_temp,Scu_temp,Scv_temp,cw_c_temp,ccw_c_temp = [],[],[],[],[],[]
    for z in range(d):
        # get spectrogram for PSD and rotary   
        u_f, u_t, u_Sxx = sig.spectrogram(um[i][z], fs=fs, window=win, \
                                                            nperseg = nps, noverlap = overlap, return_onesided=True)
        v_f, v_t, v_Sxx = sig.spectrogram(vm[i][z], fs=fs, window=win, \
                                                            nperseg = nps, noverlap = overlap, return_onesided=True)
        
        # get quadrature spectrogram and rotary
        uv_f,uv_t,uv_Cxy = spectrocross(vm[i][z],um[i][z],fs=fs,window=win,nperseg=nps,noverlap=overlap,nfft=None,detrend='constant',
                    return_onesided=True,scaling='density',axis=-1,mode='psd')     # cross spectrogram
        Sxyuv = uv_Cxy.imag                             # quadrature spectra is imag part of cross spectra
        Scw = ((u_Sxx + v_Sxx) - (2*Sxyuv)) / 2         # rotatory components
        Sccw = ((u_Sxx + v_Sxx) + (2*Sxyuv)) / 2 
        
        # get high-res for spectrogram and depth-band plots (finer resolution)
        u_f_s, u_t_s, u_Sxx_s = sig.spectrogram(um[i][z], fs=fs, window=win, \
                                                            nperseg = nps_s, noverlap = overlap_s, return_onesided=True)
        v_f_s, v_t_s, v_Sxx_s = sig.spectrogram(vm[i][z], fs=fs, window=win, \
                                                            nperseg = nps_s, noverlap = overlap_s, return_onesided=True)
        
        # get high-res quadrature spectrogram and rotary
        uv_f_s,uv_t_s,uv_Cxy_s = spectrocross(vm[i][z],um[i][z],fs=fs,window=win,nperseg=nps_s,noverlap=overlap_s,nfft=None,
                                              detrend='constant',return_onesided=True,scaling='density',
                                              axis=-1,mode='psd')     # cross spectrogram
        Sxyuv_s = uv_Cxy_s.imag                                 # quadrature spectra is imag part of cross spectra
        Scw_s = ((u_Sxx_s + v_Sxx_s) - (2*Sxyuv_s)) / 2         # rotatory components
        Sccw_s = ((u_Sxx_s + v_Sxx_s) + (2*Sxyuv_s)) / 2 
        
        # get high freq res for spectrogram and depth-band plots (finer freq resolution)
        u_f_c, u_t_c, u_Sxx_c = sig.spectrogram(um[i][z], fs=fs, window=win, \
                                                            nperseg = nps_c, noverlap = overlap_c, return_onesided=True)
        v_f_c, v_t_c, v_Sxx_c = sig.spectrogram(vm[i][z], fs=fs, window=win, \
                                                            nperseg = nps_c, noverlap = overlap_c, return_onesided=True)
        
        # get high freq quadrature spectrogram and rotary
        uv_f_c,uv_t_c,uv_Cxy_c = spectrocross(vm[i][z],um[i][z],fs=fs,window=win,nperseg=nps_c,noverlap=overlap_c,nfft=None,
                                              detrend='constant',return_onesided=True,scaling='density',
                                              axis=-1,mode='psd')     # cross spectrogram
        Sxyuv_c = uv_Cxy_c.imag                                 # quadrature spectra is imag part of cross spectra
        Scw_c = ((u_Sxx_c + v_Sxx_c) - (2*Sxyuv_c)) / 2         # rotatory components
        Sccw_c = ((u_Sxx_c + v_Sxx_c) + (2*Sxyuv_c)) / 2 
        
        # get smooth highpass data for tidal analysis
        u_f_SN, u_t_SN, u_Sxx_SN = sig.spectrogram(uh[i][z], fs=fs, window=win, \
                                                            nperseg = nps_SN, noverlap = overlap_SN, return_onesided=True)
        v_f_SN, v_t_SN, v_Sxx_SN = sig.spectrogram(vh[i][z], fs=fs, window=win, \
                                                            nperseg = nps_SN, noverlap = overlap_SN, return_onesided=True)
        
        # get tidal quadrature spectrogram and rotary
        uv_f_SN,uv_t_SN,uv_Cxy_SN = spectrocross(vh[i][z],uh[i][z],fs=fs,window=win,nperseg=nps_SN,noverlap=overlap_SN,nfft=None,
                                              detrend='constant',return_onesided=True,scaling='density',
                                              axis=-1,mode='psd')     # cross spectrogram
        Sxyuv_SN = uv_Cxy_SN.imag                                 # quadrature spectra is imag part of cross spectra
        Scw_SN = ((u_Sxx_SN + v_Sxx_SN) - (2*Sxyuv_SN)) / 2         # rotatory components
        Sccw_SN = ((u_Sxx_SN + v_Sxx_SN) + (2*Sxyuv_SN)) / 2 

        # convert spectro_t to datetime for x-axis on plots for PSD
        spectro_t4 = u_t*fs
        spectro_time_len = len(spectro_t4)
        spectro_time_axis = np.zeros([spectro_time_len],dtype='datetime64[s]')
        for k in range(spectro_time_len):
            j = int(spectro_t4[k])
            spectro_time_axis[k] = ds[i].t_seg[j].values    
            
        # convert spectro_t to datetime for x-axis on plots for spectro/depth-band
        spectro_t4_s = u_t_s*fs
        spectro_time_len_s = len(spectro_t4_s)
        spectro_time_axis_s = np.zeros([spectro_time_len_s],dtype='datetime64[s]')
        for k in range(spectro_time_len_s):
            j = int(spectro_t4_s[k])
            spectro_time_axis_s[k] = ds[i].t_seg[j].values
            
        # convert spectro_t to datetime for x-axis on plots for high-freq resolution (continuum)
        spectro_t4_c = u_t_c*fs
        spectro_time_len_c = len(spectro_t4_c)
        spectro_time_axis_c = np.zeros([spectro_time_len_c],dtype='datetime64[s]')
        for k in range(spectro_time_len_c):
            j = int(spectro_t4_c[k])
            spectro_time_axis_c[k] = ds[i].t_seg[j].values
            
        # convert spectro_t to datetime for x-axis on plots for tidal analysis
        spectro_t4_SN = u_t_SN*fs
        spectro_time_len_SN = len(spectro_t4_SN)
        spectro_time_axis_SN = np.zeros([spectro_time_len_SN],dtype='datetime64[s]')
        for k in range(spectro_time_len_SN):
            j = int(spectro_t4_SN[k])
            spectro_time_axis_SN[k] = ds[i].t_seg[j].values
        
        spect_SN_temp.append(spectro_time_axis_SN)
        specf_SN_temp.append(u_f_SN)
        cw_SN_temp.append(Scw_SN)
        ccw_SN_temp.append(Sccw_SN)
        Sduh_temp.append(u_Sxx_SN)
        Sdvh_temp.append(v_Sxx_SN)
        ###
        spect_temp.append(spectro_time_axis)
        specf_temp.append(u_f)
        cw_temp.append(Scw)
        ccw_temp.append(Sccw)
        cw_s_temp.append(Scw_s)
        ccw_s_temp.append(Sccw_s)
        spect_s_temp.append(spectro_time_axis_s)
        specf_s_temp.append(u_f_s)
        cw_c_temp.append(Scw_c)
        ccw_c_temp.append(Sccw_c)
        spect_c_temp.append(spectro_time_axis_c)
        specf_c_temp.append(u_f_c)
        Sxu_temp.append(u_Sxx)         # non-whitened
        Sxv_temp.append(v_Sxx)
        Sdu_temp.append(u_Sxx_s)
        Sdv_temp.append(v_Sxx_s)
        Scu_temp.append(u_Sxx_c)
        Scv_temp.append(v_Sxx_c)
        
    spect_SN.append(spect_SN_temp[0]) 
    specf_SN.append(specf_SN_temp[0])
    cw_SN.append(cw_SN_temp)
    ccw_SN.append(ccw_SN_temp)
    Sduh.append(Sduh_temp)          # non-whitened spectro/depth-band
    Sdvh.append(Sdvh_temp)          # list[array_segment][depth][frequency][time_segment]
    
    spect.append(spect_temp[0])   # append arrays to lists (same times and frequencies at each depth)
    specf.append(specf_temp[0])
    spect_s.append(spect_s_temp[0])
    specf_s.append(specf_s_temp[0])
    spect_c.append(spect_c_temp[0])
    specf_c.append(specf_c_temp[0])
    Sxu.append(Sxu_temp)          # PSD
    Sxv.append(Sxv_temp)
    cw.append(cw_temp)            # rotary spectra
    ccw.append(ccw_temp)
    cw_s.append(cw_s_temp)
    ccw_s.append(ccw_s_temp)
    cw_c.append(cw_c_temp)
    ccw_c.append(ccw_c_temp)
    Sdu.append(Sdu_temp)          # non-whitened spectro/depth-band
    Sdv.append(Sdv_temp)          # list[array_segment][depth][frequency][time_segment]
    Scu.append(Scu_temp)          # non-whitened spectro/depth-band
    Scv.append(Scv_temp)          # list[array_segment][depth][frequency][time_segment]

## Save
Save key values and arrays to NetCDF format using xarray.

In [8]:
# save to .nc files

for i in range(n_seg):
    ds_out = xr.Dataset( 
        data_vars=dict(
            Sxxu=(['depth','specf','spect'], Sxu[i]),                   # depth-freq-time data for PSD at each depth
            Sxxv=(['depth','specf','spect'], Sxv[i]),
            cw=(['depth','specf','spect'], cw[i]),                      # depth-freq-time data for rotary at each depth
            ccw=(['depth','specf','spect'], ccw[i]),
            Sxxu_d=(['depth','specf_s','spect_s'], Sdu[i]),             # depth-freq-time data for depth-band/spectro at each depth
            Sxxv_d=(['depth','specf_s','spect_s'], Sdv[i]),
            cw_d=(['depth','specf_s','spect_s'], cw_s[i]),              # depth-freq-time data for rotary for depth-band/spectro
            ccw_d=(['depth','specf_s','spect_s'], ccw_s[i]),
            Sxxu_c=(['depth','specf_c','spect_c'], Scu[i]),             # depth-freq-time data for depth-band/spectro at each depth
            Sxxv_c=(['depth','specf_c','spect_c'], Scv[i]),
            cw_c=(['depth','specf_c','spect_c'], cw_c[i]),              # depth-freq-time data for rotary for depth-band/spectro
            ccw_c=(['depth','specf_c','spect_c'], ccw_c[i]),
            Sxxu_SN=(['depth','specf_SN','spect_SN'], Sduh[i]),             # depth-freq-time data for tidal analysis
            Sxxv_SN=(['depth','specf_SN','spect_SN'], Sdvh[i]),
            cw_SN=(['depth','specf_SN','spect_SN'], cw_SN[i]),              # depth-freq-time data for rotary for tidal analysis
            ccw_SN=(['depth','specf_SN','spect_SN'], ccw_SN[i]),
            WKB_scale=(['depth'],int_scale),                            # WKB multiplier
        ),
        coords=dict(
            depth=depth,                               # depth values
            specf=specf[i],                            # PSD frequency bins
            specf_s=specf_s[i],                        # spectrogram frequency bins
            specf_c=specf_c[i],                        # continuum frequency bins
            specf_SN=specf_SN[i],                      # tidal freq bins
            spect=spect[i],                            # PSD time bins
            spect_s=spect_s[i],                        # spectrogram time bins
            spect_c=spect_c[i],                        # continuum time bins
            spect_SN=spect_SN[i],                      # tidal time bins
        ),
        attrs=dict(
            description=f'Depth-freq-time data for {adcp} {t_stamp} segment {i} (from 0 to {n_seg - 1}).',
            n_seg=n_seg,                               # n_seg for major datagaps each year
            adcp=adcp,                                 # adcp
            t_stamp=t_stamp,                           # time stamp
            t=time_total,                              # length of initial time series
            d=d,                                       # depth range
            start_date=str(ds[i].t_seg[0].values),     # start date of initial time series segment
            end_date=str(ds[i].t_seg[-1].values),      # end date of initial time series segment
            fs=fs,                                     # sampling frequency
            window=win,                                # fft window
            nps=nps,                                   # fft segment length
            overlap=overlap,                           # fft window overlap
            nps_s=nps_s,                               # fft segment length for depth-band and spectro
            overlap_s=overlap_s,                       # fft window overlap for depth-band and spectro
            nps_c=nps_c,                               # fft segment length for continuum
            overlap_c=overlap_c,                       # fft window overlap for continuum
            nps_SN=nps_SN,                             # fft segment length for tidal
            overlap_SN=overlap_SN,                     # fft window length for tidal
        ),
    ) 
    if adcp == 'Shoulder':
        ds_out.to_netcdf(f'../Data/data/spectra/spectra_{adcp2}_1min_{t_stamp}_{i}.nc')
    else:
        ds_out.to_netcdf(f'../Data/data/spectra/spectra_{adcp}_{t_stamp}_{i}.nc')